# 东方财富网国内新闻数据抓取器
## 拂晓工作室
### 此程序将爬取某个板块所有最近的新闻并存进CSV！

# 参考资料
* [1] BeautifulSoup的例子：https://qiita.com/rusarusa/items/d7f014ba80d6fe7a3e07
* [2] BeautifulSoup的文档：https://www.cnblogs.com/zhaof/p/6930955.html
* [3] How to install BS4 aggressively: https://github.com/ipython/ipython/issues/10684

In [6]:
# 导入库
import urllib
import re
import pandas as pd
import pymysql
import os
import turicreate
import bs4


In [4]:
# 新闻板块
china = 'cgnjj'  # 国内
international = 'cgjjj'  # 国际

# 定义基本常量
url = 'http://finance.eastmoney.com/news/' + china + '.html' # 主目录地址 更换不同主目录以爬取不同板块内容 (BAD)
csv_filepath = '../Datasets/Eastmoney/news/'  # 存储数据文件地址

In [12]:
# 创建一个二十五页的list
page_list = []
counter = 1

while counter <= 25:
    pageurl = 'http://finance.eastmoney.com/news/' + china
    if counter != 1:
        pageurl = pageurl + '_' + str(counter) + '.html'
        page_list.append(pageurl)
    else:
        pageurl = pageurl + '.html'
        page_list.append(pageurl)
    counter += 1
    
print('成功创建包含 ' + str(len(page_list)) + ' 个页面链接的目录！')

成功创建包含 25 个页面链接的目录！


In [18]:
# 创建一个包含20个class的list
counter = 0
class_list = []

while counter < 20:
    class_list.append('newsTr' + str(counter))
    counter += 1
    
print('成功创建包含 ' + str(len(class_list)) + ' 个class的目录！')

成功创建包含 20 个class的目录！


In [ ]:
# 抓取新闻URL函数
news_list = []





In [ ]:
# 抓取新闻内容函数
def getContents(html):
    

In [4]:
# 爬虫抓取网页函数 
def getHtml(url):
    html = urllib.request.urlopen(url).read()
    html = html.decode('gbk')
    return html


# 抓取网页股票代码函数 (不使用)
def getStackCode(html):
    s = r'<li><a target="_blank" href="http://quote.eastmoney.com/\S\S(.*?).html">'
    pat = re.compile(s)
    code = pat.findall(html)
    return code

# 抓取网页所有最近（25）内相关内容的函数
def getNewsList(url):
    urllist = []
    template = r'<li><a '
    return urllist